# AI Studio resource creation

In this notebook, you will create all the necessary resources for AI Studio including:

* Hub
* Project
* AI Services resource
* AI Services connection to hub
* Model deployment
* AI Search resource
* AI Search connection to hub

Requirements:
* Azure subscription

## Prerequisites

You'll need your Subscription Id to configure the ML Client.

In [ ]:
from azure.identity import DefaultAzureCredential

my_subscription_id = "{SUBSCRIPTION_ID}"
my_credentials = DefaultAzureCredential()

my_resource_group_name = ""
my_location = "East US"

In [ ]:
# Create a resource group
from azure.mgmt.resource import ResourceManagementClient

resource_client = ResourceManagementClient(my_credentials, my_subscription_id)

my_resource_group = resource_client.resource_groups.create_or_update(my_resource_group_name, {"location": my_location})

# Create the ML CLient
from azure.ai.ml import MLClient

ml_client = MLClient(
    resource_group_name=my_resource_group_name, subscription_id=my_subscription_id, credential=my_credentials
)

### Naming
Input your preferred named for the resources that will be created.

In [ ]:
## Hub
my_hub_name = "myexamplehub"
my_hub_display_name = "My Example Hub"

## Project
my_project_name = ""
my_project_location = my_location
my_project_display_name = "My Example Project"

## AI Services
my_aiservices_resource_name = "myexampleaiservices"
my_aisearch_resource_name = "sdktest-aisearch"
my_aisearch_index_name = "aisearch_index"
my_azure_openai_chat_model = "gpt-35-turbo"
my_azure_openai_embedding_model = "text-embedding-ada-002"

## Connection
my_connection_name = "myaiservivce"

## Hub creation

### Create a basic hub

A basic hub will automatically create an Azure Key Vault and Azure Storage.

If you would like to include an existing Azure Key Vault and Azure Storage, make sure to add the additional fields:
* storage_account = "{existing_storage_ARM_id}"
* key_vault = "{existing_keyvault_ARM_id}"

In [ ]:
from azure.ai.ml.entities import Hub

# construct a basic hub
my_hub = Hub(name=my_hub_name, location=my_location, display_name=my_hub_display_name)

created_hub = ml_client.workspaces.begin_create(my_hub).result()

## Project creation

A project will be created and associated to the hub we just created.

In [ ]:
from azure.ai.ml.entities import Project

# construct the project
my_project = Project(
    name=my_project_name, location=my_project_location, display_name=my_project_display_name, hub_id=created_hub.id
)

created_project = ml_client.workspaces.begin_create(workspace=my_project).result()

## AI Services resource creation

In [ ]:
# Create AI Services resource, this may take a few minutes

from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.ai.ml import MLClient
from azure.ai.ml.entities import AzureAIServicesConnection

aiservices_client = CognitiveServicesManagementClient(credential=my_credentials, subscription_id=my_subscription_id)

aiservices_account = aiservices_client.accounts.begin_create(
    resource_group_name=my_resource_group_name,
    account_name=my_aiservices_resource_name,
    account={
        "sku": {"name": "S0"},
        "kind": "AIServices",
        "location": my_location,
        "properties": {"custom_sub_domain_name": my_aiservices_resource_name},
    },
).result()

## AI Services Connection

In [ ]:
aiservices_key = aiservices_client.accounts.list_keys(
    resource_group_name=my_resource_group_name,
    account_name=my_aiservices_resource_name,
).key1

aiservices_endpoint = aiservices_client.accounts.get(
    resource_group_name=my_resource_group_name, account_name=my_aiservices_resource_name
).properties.endpoint

resource_id = f"/subscriptions/{my_subscription_id}/resourceGroups/{my_resource_group_name}/providers/Microsoft.CognitiveServices/accounts/{my_aiservices_resource_name}"
aiservices_connection = AzureAIServicesConnection(
    name="azure_aiservices_connection",
    api_key=aiservices_key,
    endpoint=aiservices_endpoint,
    ai_services_resource_id=resource_id,
)

ml_client = MLClient(
    workspace_name=my_hub_name,
    resource_group_name=my_resource_group_name,
    subscription_id=my_subscription_id,
    credential=my_credentials,
)

result = ml_client.connections.create_or_update(connection=aiservices_connection)

# Create Model Deployments

In [ ]:
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

client = CognitiveServicesManagementClient(
    credential=my_credentials,
    subscription_id=my_subscription_id,
)

deployment = client.deployments.begin_create_or_update(
    resource_group_name=my_resource_group_name,
    deployment_name=my_azure_openai_chat_model,
    account_name=my_aiservices_resource_name,
    deployment={
        "properties": {
            "model": {
                "format": "OpenAI",
                "name": my_azure_openai_chat_model,
                "version": "0301",
            }
        },
        "sku": {"capacity": 50, "name": "Standard"},
    },
).result()

deployment = client.deployments.begin_create_or_update(
    resource_group_name=my_resource_group_name,
    deployment_name=my_azure_openai_embedding_model,
    account_name=my_aiservices_resource_name,
    deployment={
        "properties": {
            "model": {
                "format": "OpenAI",
                "name": my_azure_openai_embedding_model,
                "version": "2",
            }
        },
        "sku": {"capacity": 50, "name": "Standard"},
    },
).result()

## Create AI Search resource and connection

In [ ]:
from azure.mgmt import search as Se
from azure.ai.ml.entities import ApiKeyConfiguration, AzureAISearchConnection

# Create AI Search resource
search_client = Se.SearchManagementClient(credential=my_credentials, subscription_id=my_subscription_id)

search = search_client.services.begin_create_or_update(
    resource_group_name=my_resource_group_name,
    search_service_name=my_aisearch_resource_name,
    service={
        "location": my_location,
        "sku": {"name": "standard"},
        # "properties": {"hostingMode": "default", "partitionCount": 1, "replicaCount": 3},
    },
).result()

# create connection to the hub
aisearch_key = search_client.admin_keys.get(my_resource_group_name, my_aisearch_resource_name).primary_key
aisearch_endpoint = f"https://{my_aisearch_resource_name}.search.windows.net"

aisearch_connection = AzureAISearchConnection(
    name="azure_aisearch_connection",
    endpoint=aisearch_endpoint,
    credentials=ApiKeyConfiguration(key=aisearch_key),
)

# Create an ML client
ml_client = MLClient(
    workspace_name=my_hub_name,
    resource_group_name=my_resource_group_name,
    subscription_id=my_subscription_id,
    credential=my_credentials,
)

ml_client.connections.create_or_update(aisearch_connection)

## Update the ML Client

We will input the newly created project into the MLClient to use it.

In [ ]:
ml_client = MLClient(
    workspace_name=my_project_name,
    resource_group_name=my_resource_group_name,
    subscription_id=my_subscription_id,
    credential=DefaultAzureCredential(),
)